In [ ]:
import numpy as np
import healpy as hp
from pathlib import Path

In [ ]:
datadir=Path("data/")

In [ ]:
output_nside = 4096

In [ ]:
output_lmax = min(3*output_nside - 1, 8192*2)

## Large scales

In [ ]:
alm_log_pol_tens_large_scale = hp.read_alm(datadir / "gnilc_dust_largescale_template_logpoltens_alm_nside2048_lmax3072_complex64.fits", hdu=(1,2,3))

In [ ]:
map_log_pol_tens_large_scale = hp.alm2map(alm_log_pol_tens_large_scale.astype(np.complex128), nside=output_nside)

## Small scales modulation

In [ ]:
modulate_alm = { k:hp.read_alm(datadir/f"gnilc_dust_{k}_modulation_alms_lmax3072.fits").astype(np.complex128) for k in ["temperature","polarization"] }

## Small scales

In [ ]:
cl_small_scale = hp.read_cl(datadir / "gnilc_dust_small_scales_logpoltens_cl_lmax16384.fits")

In [ ]:
synalm_lmax = 8192*2 # for reproducibility
# synalm_lmax = output_lmax
np.random.seed(8192)

alm_log_pol_tens_small_scale = hp.synalm(
    list(cl_small_scale) + [np.zeros_like(cl_small_scale[0])] * 3,
    lmax=synalm_lmax,
    new=True,
)

alm_log_pol_tens_small_scale = [hp.almxfl(each, np.ones(3*output_nside-1)) for each in alm_log_pol_tens_small_scale]
map_log_pol_tens_small_scale = hp.alm2map(alm_log_pol_tens_small_scale, nside=output_nside)
map_log_pol_tens_small_scale[0] *= hp.alm2map(modulate_alm["temperature"], output_nside)
map_log_pol_tens_small_scale[1:] *= hp.alm2map(modulate_alm["polarization"], output_nside)
assert np.isnan(map_log_pol_tens_small_scale).sum() == 0

## Combine scales

* Combine small and large scale maps
* Transform from logpoltens to IQU
* Write output map

In [ ]:
map_log_pol_tens = map_log_pol_tens_large_scale + map_log_pol_tens_small_scale

In [ ]:
from log_pol_tens_utils import log_pol_tens_to_map

In [ ]:
output_map = log_pol_tens_to_map(map_log_pol_tens)

Planck 353 GHz color correction https://github.com/galsci/pysm/issues/99

In [ ]:
output_map *= 0.911

In [ ]:
hp.write_map(datadir / f"gnilc_dust_template_nside{output_nside}_float32.fits", output_map, dtype=np.float32, overwrite=True)